In [16]:
import pprint as pp

artists = []
titles = []
with open('data/top100_july2019.txt') as f:
    lines = f.readlines()
    
    for line in lines:
        titles.append(line.split('-')[0].strip())
        artists.append(line.split('-')[1].strip())

['Blackpink',
 'MOMOLAND',
 'Bigbang',
 'RED VELVET',
 'Bigbang',
 'Bigbang',
 'MOMOLAND',
 'BTS',
 'Bigbang',
 'Super Junior',
 'Blackpink',
 'BTS',
 'BTS ft. Halsey',
 'Hyuna',
 'EXO',
 'Twice',
 '4Minute',
 'G',
 'PSY',
 'ITZY',
 'TWICE',
 'BTS',
 'Du Ddu',
 'BTS',
 'Seventeen',
 'BTS',
 'Red Velvet',
 'f(x)',
 'Wanna One',
 'Taeyang',
 'BTS',
 'TWICE',
 'Bigbang',
 'BTS',
 'Bigbang',
 'PSY',
 "Girls' Generation",
 'PSY',
 'GD x Taeyang',
 'EXO',
 'Bigbang',
 'TWICE',
 'Taeyeon ft. Verbal Jint',
 '2NE1',
 "Girls' Generation",
 'BTS',
 'Hyuna',
 'BTS',
 'GOT7',
 'BTS',
 'GOT7',
 'iKON',
 'BLACKPINK',
 'Twice',
 'EXO',
 '(G)I',
 'Bigbang',
 'AHH',
 'Twice',
 'Bigbang',
 'EXO',
 'EXO',
 'iKON',
 'EXO',
 'BTS',
 'EXO',
 'Super Junior',
 'BTS',
 'GOT7',
 'PSY',
 'BTS',
 "Girls' Generation",
 'PSY ft. Hyuna',
 'EXO',
 'IU ft. G',
 'A',
 'Blackpink',
 'WINNER',
 'RED VELVET',
 'Taeyang',
 'SHINee',
 'BTS',
 'Red Velvet',
 'BTS',
 'PENTAGON',
 'TWICE',
 'Bigbang',
 'JENNIE',
 'BTS',
 'BLACK

In [17]:
from requests import get
from requests.exceptions import RequestException
from contextlib import closing
from bs4 import BeautifulSoup

In [39]:
def fetch(url):
    try:
        with closing(get(url, stream=True)) as resp:
            if validate(resp):
                return resp.content
            else:
                return None
    except RequestException as e:
        log_error('Error during requests to {0} : {1}'.format(url, str(e)))
        return None

In [40]:
def validate(resp):
    content_type = resp.headers['Content-Type'].lower()
    return (resp.status_code == 200
           and content_type is not None
           and content_type.find('html') > -1)

In [41]:
def log_error(e):
    print(e)

In [42]:
import string

def generate_query_string(artist, title):
    artist_q = artist.translate(
        str.maketrans(
            '',
            '',
            string.punctuation)
    ).lower().replace(' ', '-')
    title_q = title.translate(
        str.maketrans(
            '', 
            '', 
            string.punctuation)
    ).lower().replace(' ', '-')
    
    return artist_q + "-" + title_q

In [80]:
def extract_lyrics(artists, titles):
    for artist, title in zip(artists, titles):
        url = "https://genius.com/" + generate_query_string(artist, title) + "-lyrics"
        
        raw_html = fetch(url)
        if raw_html is not None:
            html = BeautifulSoup(raw_html, 'html.parser')
            lyricdivs = html.findAll('div', {'class': 'lyrics'})
            for div in lyricdivs:
                lyrics = div.find('p')

                with open("./data/lyric_files/{0}-lyrics.txt".format(generate_query_string(artist, title)), 'w') as f:
                    f.write(lyrics.text)
        else:
            print("lyrics not found for {0}, {1}".format(artist, title))

In [81]:
extract_lyrics(artists, titles)

lyrics not found for BTS, Blood, Sweat & Tears
lyrics not found for BTS ft. Halsey, Boy With Luv
lyrics not found for G, Crooked
lyrics not found for Du Ddu, Ddu
lyrics not found for Taeyang, Eyez, Nose, Lips
lyrics not found for Taeyeon ft. Verbal Jint, I
lyrics not found for (G)I, LATATA
lyrics not found for AHH, Like OOH
lyrics not found for PSY ft. Hyuna, Oppa Is Just My Style
lyrics not found for IU ft. G, Palette
lyrics not found for A, Peek
lyrics not found for SHINee, RingDingDong
lyrics not found for PENTAGON, Shine
lyrics not found for BTS, We Are Bulletproof
lyrics not found for G, Who, You?


In [85]:
def manual_lyric_extraction():
    urls = [
        "https://genius.com/Bts-blood-sweat-and-tears-lyrics",
        "https://genius.com/Bts-boy-with-luv-lyrics",
        "https://genius.com/Blackpink-ddu-du-ddu-du-lyrics",
        "https://genius.com/G-i-dle-latata-lyrics",
    ]
    
    for url in urls:
        raw_html = fetch(url)
        if raw_html is None:
            print("unable to fetch lyrics for {0}".format(url))
        
        html = BeautifulSoup(raw_html, 'html.parser')
        lyricdivs = html.findAll('div', {'class': 'lyrics'})
        for div in lyricdivs:
            lyrics = div.find('p')
            
            
            with open("./data/lyric_files/{0}".format(url[19:]), 'w') as f:
                f.write(lyrics.text)

In [86]:
manual_lyric_extraction()